In [ ]:
# (1) 데이터 가져오기
# - 터미널에서 ~/data/data/bike-sharing-demand 경로에 train.csv 데이터를 train 변수로 가져옵니다.

In [ ]:
# (2) datetime 컬럼을 datetime 자료형으로 변환하고 연, 월, 일, 시, 분, 초까지 6가지 컬럼 생성하기
# - to_datetime 활용
# - pandas datetime extract year month day 키워드로 검색해 보기

In [ ]:
# (3) year, month, day, hour, minute, second 데이터 개수 시각화하기
# - sns.countplot 활용해서 시각화하기
# - subplot을 활용해서 한 번에 6개의 그래프 함께 시각화하기

In [ ]:
# (4) X, y 컬럼 선택 및 train/test 데이터 분리
# - X (feature) 에는 어떤 데이터 컬럼을 넣어야 될지 고민해 봅시다. 데이터 컬럼을 추가하거나 빼면서 잘 조합하여 데이터를 넣기 바랍니다. 만일 모든 데이터 컬럼(특징)을 넣는다면 오차 값이 말도 안 되게 적게 나올 수도 있습니다
# - ex) count를 예측하는데 casual, registered 컬럼이 필요할까요?
# - count 값을 맞추고자 하므로, y 변수에 count 컬럼의 데이터 넣기

In [ ]:
# (5) LinearRegression 모델 학습
# - sklearn의 LinearRegression 모델 불러오기 및 학습하기

In [ ]:
# (6) 학습된 모델로 X_test에 대한 예측값 출력 및 손실함수값 계산
# - 학습된 모델에 X_test를 입력해서 예측값 출력하기
# - 모델이 예측한 값과 정답 target간의 손실함수 값 계산하기
# - mse 값과 함께 rmse 값도 계산하기

In [ ]:
# (7) x축은 temp 또는 humidity로, y축은 count로 예측 결과 시각화하기
# - x축에 X 데이터 중 temp 데이터를, y 축에는 count 데이터를 넣어서 시각화하기
# - x축에 X 데이터 중 humidity 데이터를, y 축에는 count 데이터를 넣어서 시각화하기